In [ ]:
import numpy as np
import os

DIR = os.path.join(os.path.expanduser('~'))
animal = 'MD589_kui'
volume_path = os.path.join(DIR, f'{animal}_annotations.npy')

# Numpy array whose shape should match (x, y, z)
with open(volume_path, 'rb') as file:
    volume = np.load(file)
volume = np.swapaxes(volume, 0, 1)

# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
resol = (14464, 14464, 20000)

# Voxel offset
offset = (0, 0, 0)

# Layer type
layer_type = ['segmentation'][0]

In [ ]:
# Run this cell if you want to preview the numpy array before converting to precomputed format
import neuroglancer
viewer = neuroglancer.Viewer()
print(viewer)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=resol), 
        voxel_offset=offset
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:
# https://github.com/seung-lab/cloud-volume
# Don't change all compress=False. Neuroglancer can only read with compress=False
from cloudvolume import CloudVolume

precompute_path = os.path.join(DIR, f'{animal}_annotations')
cloudpath = f'file://{precompute_path}'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = resol, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = [256, 256, 64], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.info['segment_properties'] = 'names'
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

In [ ]:
# https://github.com/seung-lab/igneous
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

tq = LocalTaskQueue(parallel=True)
tasks = tc.create_downsampling_tasks(cloudpath, compress=False) # Downsample the volumes 
tq.insert(tasks)
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False) # The first phase of creating mesh
tq.insert(tasks)
tq.execute()

In [ ]:
# It should be able to incoporated to above cell, but it will give a weird bug. Don't know the reason
tq = LocalTaskQueue(parallel=True)
tasks = tc.create_mesh_manifest_tasks(cloudpath) # The second phase of creating mesh
tq.insert(tasks)
tq.execute()